In [1]:
import os
import numpy as np
import itertools
import trimesh
import math
import k3d
from time import sleep
from tqdm import tqdm_notebook as tqdm

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from geometry.model import Model, combine_observations, get_mesh
from geometry.utils.visualisation import illustrate_points, illustrate_mesh


# !conda install -c conda-forge pyembree
# !conda install -c conda-forge igl
# !pip install Cython
# !pip install gym

In [284]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

from geometry.model import Model, combine_observations, get_mesh
from geometry.utils.visualisation import illustrate_points, illustrate_mesh


def generate_observations(model):
    observations = []
    for view_point_idx in range(len(model.view_points)):
        observation = model.get_observation(view_point_idx)
        observations.append(set(observation.face_indexes))
    return observations

def find_greedy_optimal(model, area_threshold=0.95, do_rec=False):
    observations = generate_observations(model)

    uncovered = set(np.arange(0, model.mesh.faces.shape[0]))
    covered = set()

    uncovered_area_threshold = len(uncovered) * (1 - area_threshold)

    view_points = []
    while len(uncovered) > uncovered_area_threshold and \
            len(view_points) < len(observations):
        view_point_index = np.argmax([len(covered.union(ob)) for ob in observations])
        new = observations[view_point_index]
        covered = covered.union(new)
        uncovered -= new
        view_points.append(view_point_index)

    if not do_rec:
        return view_points, 0.0

    combined_observation = None
    for vp in view_points:
        observation = model.get_observation(vp)
        if combined_observation is None:
            combined_observation = observation
        else:
            combined_observation += observation

    reconstructed_vertices, reconstructed_faces = get_mesh(combined_observation, depth=10)
    loss = model.surface_similarity(reconstructed_vertices, reconstructed_faces)

    return view_points, loss, reconstructed_vertices, reconstructed_faces

In [295]:
model_path = "./data/00070090_73b2f35a88394199b6fd1ab8_003.obj"
model = Model(model_path, resolution_image=1024)
model.generate_view_points(100)
optimal, loss, reconstructed_vertices, reconstructed_faces = find_greedy_optimal(model, do_rec=True)

KeyboardInterrupt: 

In [288]:
optimal

[93, 0, 76, 16]

In [296]:
model = Model(model_path="./data/00070090_73b2f35a88394199b6fd1ab8_003.obj")
# model.generate_view_points(10)

model.illustrate()

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…

In [297]:
from geometry.voxel_grid import *
builder = VoxelGridBuilder(32)


In [298]:
mesh_grid = builder.build(model.mesh.vertices, model.bounds)

In [299]:
plot = k3d.plot()
plt_voxels = k3d.voxels(mesh_grid.surface_grid * 3)
plot += plt_voxels
plot

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…

In [10]:
mesh_grid.illustrate()

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…

In [ ]:
mesh_grid.surface_grid

In [52]:
from rl.environment import *

class VoxelGridWrapper(gym.ObservationWrapper):
    def __init__(self, env, grid_size=64, occlusion_reward=False):
        super().__init__(env)

        self.grid_shape = ((grid_size, grid_size, grid_size))
        self.observation_space = spaces.Box(0, 1, self.grid_shape, dtype=bool)

        self.builder = VoxelGridBuilder(grid_size)

        self.mesh_grid = None
        self.gt_size = None
        self.bounds = None
        self.plot = k3d.plot(name='wrapper')

    def reset(self):
        observation, action = self.env.reset()
        self.bounds = self.env.model.bounds

        self.mesh_grid = self.builder.build(self.env.model.mesh.vertices, self.bounds)
        self.gt_size = np.count_nonzero(self.mesh_grid.surface_grid)

        return self.observation(observation), action
    
    def observation(self, observation):
        return self.builder.build(observation.points, self.bounds, observation.direction)

    def render(self, action, observation):
        self.plot.close()
        self.plot = illustrate_voxels(observation)
        self.plot.display()

    def final_reward(self, observation):
        return self.env.final_reward(observation)

    def step_reward(self, observation):
        intersection = np.count_nonzero(np.logical_and(self.mesh_grid.surface_grid,
                                                       observation.surface_grid))
        return intersection / self.gt_size


def create_env(model_path=None):    
    env = Environment(model_path=model_path,
    #                   similarity_threshold=similarity_threshold,
                      image_size=1024,
                      number_of_view_points=100)
    # env = CombiningObservationsWrapper(env)
    # env = StepPenaltyRewardWrapper(env, weight=1.0)
    # env = DepthMapWrapper(env)

    env = MeshReconstructionWrapper(env, reconstruction_depth=8, final_depth=10, scale_factor=8,
                                    do_step_reconstruction=True)
    env = VoxelGridWrapper(env, grid_size=64)
    env = CombiningObservationsWrapper(env)
    env = VoxelWrapper(env, occlusion_reward=True)
#     env = StepPenaltyRewardWrapper(env, weight=1.0)
#     env = FrameStackWrapper(env, num_stack=4, lz4_compress=False)
#     env = ActionMaskWrapper(env)
    return env

In [127]:
from geometry.utils.sampling import generate_sunflower_sphere_points
sphere_points, phis, thetas = generate_sunflower_sphere_points(100)
sphere_points *= 80
sphere_points += 32

In [289]:
env = create_env("./data/10abc/00571302_550f5bdad5731705a2dfebd8_001.obj")

state, action = env.reset()
print(action)
view_points = [sphere_points[action][::-1]]
states = [state]


for action in [93, 0, 76, 16]:
    state, reward, done, info = env.step(action)
    states.append(state)
    view_points.append(sphere_points[action][::-1])
#     env.render(action, state)

87
0.44097720617787217 0.10155788527257448 0.5425350914504466
-3.219899023907113
0.43615433197450765 0.4828872106314756 0.9190415426059833
-1.4145432092903993
0.4357811333754378 0.5443691572810204 0.9801502906564582
-1.3042331376408547
0.4242119768042717 0.570896487678927 0.9951084644831987
-1.2460819669109169


In [293]:
plot = illustrate_voxels(states[3] * 3)
illustrate_points(view_points[:4], size=5, plot=plot)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…

In [134]:
env.model.illustrate()

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…

In [ ]:
vp = env.model.view_points[50]
g = builder.build([vp.point], env.model.mesh.bounds)
view_point = np.asarray(np.where(g.surface_grid)).reshape(-1)[::-1]

In [2]:
NUM_POINTS = 10


plot = k3d.plot(name='points')
plot.display()

model = Model("./data/Pyramid.obj", resolution_image=64)
model.generate_view_points(NUM_POINTS)

observations = []
for view_point_idx in range(NUM_POINTS):
    %time observation = model.get_observation(view_point_idx)
    
    plot = observation.illustrate(plot, size=0.04)
    plot = illustrate_points([model.get_point(view_point_idx)], size=1.0, plot=plot)
#     sleep(2)
    
    observations.append(observation)
    
combined_observation = combine_observations(observations)
%time reconstructed_vertices, reconstructed_faces = get_mesh(combined_observation)

loss = model.surface_similarity(reconstructed_vertices, reconstructed_faces)
print(loss)

# illustrate_mesh(reconstructed_vertices, reconstructed_faces)

Output()

KeyboardInterrupt: 

In [8]:
illustrate_mesh(reconstructed_vertices, reconstructed_faces)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…

In [16]:
len(combined_observation.points), loss

(755551, 0.08265528953648774)

In [9]:
len(combined_observation.points), loss

(47203, 0.2490280261287171)

In [10]:
%%time
reconstructed_vertices, reconstructed_faces = get_mesh(combined_observation)
loss = model.surface_similarity(reconstructed_vertices, reconstructed_faces)

CPU times: user 19.3 s, sys: 129 ms, total: 19.4 s
Wall time: 3.08 s


In [ ]:
NUM_POINTS = 10

models_path = "./data/1kabc/simple/val"

losses = []
for model_name in tqdm(sorted(os.listdir(models_path))):
    model = Model(os.path.join(models_path, model_name))
    model.generate_view_points(NUM_POINTS)

    combined_observation = None
    for view_point_idx in range(NUM_POINTS):
        observation = model.get_observation(view_point_idx)

        if combined_observation is None:
            combined_observation = observation
        else:
            combined_observation += observation
    
    reconstructed_vertices, reconstructed_faces = get_mesh(combined_observation)

    loss = model.surface_similarity(reconstructed_vertices, reconstructed_faces)
    print(loss)
    losses.append(loss)

0.43260879503679267
0.011748805825769773
0.6368368930715143
0.01999495878259948
0.009864535577272508
0.0049465034012368585
0.004867339413804205
2.114583249307729
5.513358045365431
0.007657026293620387
0.012763651915830367
0.022809397706465888
0.606697827210757
0.021164343424414495
0.008369065356445203
0.6542076437755041
0.008919581879375504
0.008961263241012722
0.9702387490236133
1.3275972704264303
